In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import BES survey data and BES results data 
df = pd.read_excel('custom-filtered-2026-02-18T18_48_45Z.xlsx', sheet_name='Dataset')
region = pd.read_spss('BES-2024-General-Election-results-file-v1.0.sav', usecols=["ONSConstID", "Region"]).set_index("ONSConstID")

In [3]:
# map ONS education category to simplified categories
ed_map = {
    0: 0,   # No qualification -> Low
    1: 0,   # NVQ 1 / Level 1 -> Low
    2: 1,   # GCSEs -> Intermediate
    3: 1,   # A-Levels -> Intermediate
    4: 2,   # Degrees/HNDs -> Higher
    5: 2,   # Professional/Other -> Higher
    -8: 0   # Missing/Refused -> Baseline Low
}

# keep only age 18+ populations and relevant columns
df_final = df[df['Age (14 categories) Code']>2][['Post-2019 Westminster Parliamentary constituencies Code','Age (14 categories) Code','Highest level of qualification (7 categories) Code','Sex (2 categories) Code','Observation']]

df_final.rename(columns={'Post-2019 Westminster Parliamentary constituencies Code':'ONSConstID','Age (14 categories) Code':'AgeCat','Highest level of qualification (7 categories) Code':'ONSed','Sex (2 categories) Code':'SexCat'}, inplace=True)

df_final["Region"] = df_final["ONSConstID"].map(region["Region"])

# remap education level to fewer categories
df_final['ONSed'] = df_final['ONSed'].map(ed_map)

# aggregrate education columns back into single row per constituency
df_final_agg = df_final.groupby(['ONSConstID', 'AgeCat', 'ONSed', 'SexCat', 'Region']).sum().reset_index()

In [4]:
df_final_agg.to_csv('ONS cleaned.csv')